In [9]:
import sys
sys.path.append('../')

In [10]:
import torch
import torchvision.transforms
import itertools
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

from models.train_full import train, test
from models.models import CNNClassifier, ResNetClassifier
from models.utils import set_seed

ModuleNotFoundError: No module named 'models.train_full'

In [ ]:
def pretty(ld, indent=0):
    with open('result.txt', 'w', encoding='utf-8') as file:
        for d in tqdm(ld):
            file.write('{' + '\n')
            for key, value in d.items():
                file.write('\t' * (indent+1) + str(key) + ':' + str(value) + '\n')
                # file.write('\t' * (indent+1) + str(key) + '\n')
                # file.write('\t' * (indent+2) + str(value) + '\n')
            file.write('},\n')

In [3]:
do_train = True

seed = 4444

metric_filter_1 = 'val_mcc'
metric_filter_2 = 'val_mae'

## 1

In [5]:
dict_model = dict(
    # dictionary with model information
    in_channels=[3],
    out_channels=[2],
    # dim_layers=[[32, 64, 128], [16, 32, 64]],
    dim_layers=[[32, 64, 128]],
    # block_conv_layers=[3, 5],
    block_conv_layers=[3],
    residual=[True],
    # max_pooling=[True, False],
    max_pooling=[True],
    transforms=[
        (
            '0',
            torchvision.transforms.Compose([
                torchvision.transforms.RandomResizedCrop(size=(400, 400)),
                torchvision.transforms.RandomHorizontalFlip(),
            ]),
            torchvision.transforms.Resize(size=(400, 400)),
            # transforms.RandomResizedCrop(size=(400, 400)),
            # test varios times over the same data and choose the mean
            False,
        ),
        (
            '1',
            torchvision.transforms.Compose([
                torchvision.transforms.Resize(size=(400, 400)),
                torchvision.transforms.RandomHorizontalFlip(),
            ]),
            torchvision.transforms.Resize(size=(400, 400)),
            True,
        )
    ]
)

list_model = [dict(zip(dict_model.keys(), k)) for k in itertools.product(*dict_model.values())]

In [ ]:
if do_train:
    for d in tqdm(list_model):
        set_seed(seed)
        
        d = d.copy()
        transforms = d.pop('transforms')

        train(
            model=CNNClassifier(**d),
            dict_model=d,
            log_dir="./logs_full",
            data_path="../data_full",
            save_path="./models/saved_full",
            lr=1e-2,
            optimizer_name="adamw",
            n_epochs=150,
            batch_size=32,
            num_workers=2,
            scheduler_mode='min_mse',
            debug_mode=False,
            device=None,
            steps_save=15,
            use_cpu=False,
            loss_age_weight=1e-2,
            scheduler_patience=30,
            train_transforms=transforms[1],
            test_transforms=transforms[2],
            suffix=transforms[0],
            use_cache=transforms[3],
        )

### Results

In [6]:
res_test = test(
    data_path = "../data_full",
    save_path = './models/saved_full',
    n_runs = 1,
    batch_size = 64,
    num_workers = 0,
    debug_mode = False,
    use_cpu = False,
    save = True,
    verbose = False,
    transforms=torchvision.transforms.Resize(size=(400, 400)),
    use_cache=True,
)

100%|██████████| 20/20 [54:07<00:00, 162.37s/it] 


In [13]:
all = res_test
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_1] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 3,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 135,
 'train_loss': 0.7180283,
 'val_mse_age': 131.51015,
 'train_acc': 0.9394927024841309,
 'val_acc': 0.8674965500831604,
 'val_mcc': 0.7339866991692735,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_3_True_True_0.01_adamw_32_min_mse_0.01_1_135',
 'train_mae': 5.088548,
 'val_mae': 8.730115,
 'test_mae': 8.740539,
 'train_mse': 42.900738,
 'val_mse': 131.44864,
 'test_mse': 136.33107,
 'train_mcc': 0.8789576593001115,
 'test_mcc': 0.7426456711701096,
 'test_acc': 0.8714404106140137}

In [14]:
sort_idx = np.argsort([k['dict'][metric_filter_2] for k in all])
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 3,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 120,
 'train_loss': 0.88590217,
 'val_mse_age': 127.54852,
 'train_acc': 0.9190470576286316,
 'val_acc': 0.8652835488319397,
 'val_mcc': 0.7305876856123024,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_3_True_True_0.01_adamw_32_min_mse_0.01_1_120',
 'train_mae': 5.7857547,
 'val_mae': 8.650999,
 'test_mae': 8.628361,
 'train_mse': 56.060917,
 'val_mse': 127.48625,
 'test_mse': 133.53343,
 'train_mcc': 0.8400896585159275,
 'test_mcc': 0.7315739561738387,
 'test_acc': 0.8645285964012146}

## 2

In [5]:
dict_model = dict(
    # dictionary with model information
    in_channels=[3],
    out_channels=[2],
    dim_layers=[[32, 64, 128], [16, 32, 64]],
    block_conv_layers=[3],
    residual=[True],
    max_pooling=[True, False],
    transforms=[
        (
            '1',
            torchvision.transforms.Compose([
                torchvision.transforms.Resize(size=(400, 400)),
                torchvision.transforms.RandomHorizontalFlip(),
            ]),
            torchvision.transforms.Resize(size=(400, 400)),
            True,
        ),
    ]
)

list_model = [dict(zip(dict_model.keys(), k)) for k in itertools.product(*dict_model.values())]

In [ ]:
if do_train:
    for d in tqdm(list_model):
        set_seed(seed)
        
        d = d.copy()
        transforms = d.pop('transforms')

        train(
            model=CNNClassifier(**d),
            dict_model=d,
            log_dir="./logs_full_2",
            data_path="../data_full",
            save_path="./models/saved_full_2",
            lr=1e-2,
            optimizer_name="adamw",
            n_epochs=200,
            batch_size=32,
            num_workers=3,
            scheduler_mode='min_mse',
            debug_mode=False,
            device=None,
            steps_save=15,
            use_cpu=False,
            loss_age_weight=1e-2,
            scheduler_patience=30,
            train_transforms=transforms[1],
            test_transforms=transforms[2],
            suffix=transforms[0],
            use_cache=transforms[3],
        )

### Results

In [7]:
res_test = test(
    data_path = "../data_full",
    save_path = './models/saved_full_2',
    n_runs = 1,
    batch_size = 64,
    num_workers = 0,
    debug_mode = False,
    use_cpu = False,
    save = True,
    verbose = False,
    transforms=torchvision.transforms.Resize(size=(400, 400)),
    use_cache=True,
)

100%|██████████| 26/26 [53:13<00:00, 122.81s/it] 


In [8]:
all = res_test
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_1] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 3,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 120,
 'train_loss': 0.3935543,
 'val_mse_age': 154.79866,
 'train_acc': 0.9855399131774902,
 'val_acc': 0.8511756658554077,
 'val_mcc': 0.7011677685312916,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_3_True_True_0.01_adamw_32_min_mse_0.01_1_120',
 'train_mae': 3.5678704,
 'val_mae': 9.337749,
 'test_mae': 9.498872,
 'train_mse': 20.937126,
 'val_mse': 154.50348,
 'test_mse': 162.88927,
 'train_mcc': 0.9710754062237904,
 'test_mcc': 0.7199835661808924,
 'test_acc': 0.8598285913467407}

In [11]:
all = res_test
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_2] for k in all])
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 3,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 120,
 'train_loss': 0.3935543,
 'val_mse_age': 154.79866,
 'train_acc': 0.9855399131774902,
 'val_acc': 0.8511756658554077,
 'val_mcc': 0.7011677685312916,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_3_True_True_0.01_adamw_32_min_mse_0.01_1_120',
 'train_mae': 3.5678704,
 'val_mae': 9.337749,
 'test_mae': 9.498872,
 'train_mse': 20.937126,
 'val_mse': 154.50348,
 'test_mse': 162.88927,
 'train_mcc': 0.9710754062237904,
 'test_mcc': 0.7199835661808924,
 'test_acc': 0.8598285913467407}

## 3

In [13]:
dict_model = dict(
    # dictionary with model information
    in_channels=[3],
    out_channels=[2],
    dim_layers=[[32, 64, 128], [16, 32, 64]],
    block_conv_layers=[3, 5],
    residual=[True],
    max_pooling=[True],
    transforms=[
        (
            '1',
            torchvision.transforms.Compose([
                torchvision.transforms.Resize(size=(200, 200)),
                torchvision.transforms.RandomHorizontalFlip(),
            ]),
            torchvision.transforms.Resize(size=(200, 200)),
            True,
        ),
    ]
)

list_model = [dict(zip(dict_model.keys(), k)) for k in itertools.product(*dict_model.values())]

In [8]:
if do_train:
    for d in tqdm(list_model):
        set_seed(seed)
        
        d = d.copy()
        transforms = d.pop('transforms')

        train(
            model=CNNClassifier(**d),
            dict_model=d,
            log_dir="./logs_full_3",
            data_path="../data_full",
            save_path="./models/saved_full_3",
            lr=1e-2,
            optimizer_name="adamw",
            n_epochs=150,
            batch_size=32,
            num_workers=3,
            scheduler_mode='min_mse',
            debug_mode=False,
            device=None,
            steps_save=15,
            use_cpu=False,
            loss_age_weight=1e-2,
            scheduler_patience=50,
            train_transforms=transforms[1],
            test_transforms=transforms[2],
            suffix=transforms[0],
            use_cache=transforms[3],
        )

in_channels=3/out_channels=2/dim_layers=[32/64/128]/block_conv_layers=3/residual=True/max_pooling=True/train_lr=0.01/train_optimizer_name=adamw/train_batch_size=32/train_scheduler_mode=min_mse/train_loss_age_weight=0.01/train_suffix=1 -> 142.6066436767578: 100%|██████████| 150/150 [4:32:08<00:00, 108.86s/it]
in_channels=3/out_channels=2/dim_layers=[32/64/128]/block_conv_layers=5/residual=True/max_pooling=True/train_lr=0.01/train_optimizer_name=adamw/train_batch_size=32/train_scheduler_mode=min_mse/train_loss_age_weight=0.01/train_suffix=1 -> 136.85464477539062: 100%|██████████| 150/150 [5:57:56<00:00, 143.18s/it]
in_channels=3/out_channels=2/dim_layers=[16/32/64]/block_conv_layers=3/residual=True/max_pooling=True/train_lr=0.01/train_optimizer_name=adamw/train_batch_size=32/train_scheduler_mode=min_mse/train_loss_age_weight=0.01/train_suffix=1 -> 164.13406372070312: 100%|██████████| 150/150 [2:21:44<00:00, 56.69s/it]
in_channels=3/out_channels=2/dim_layers=[16/32/64]/block_conv_layers=5

### Results

In [16]:
res_test = test(
    data_path = "../data_full",
    save_path = './models/saved_full_3',
    n_runs = 1,
    batch_size = 64,
    num_workers = 0,
    debug_mode = False,
    use_cpu = False,
    save = True,
    verbose = False,
    transforms=torchvision.transforms.Resize(size=(200, 200)),
    use_cache=True,
)

100%|██████████| 40/40 [30:18<00:00, 45.47s/it]  


In [17]:
all = res_test
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_1] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 5,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 135,
 'train_loss': 0.09937081,
 'val_mse_age': 136.1627,
 'train_acc': 0.9453597068786621,
 'val_acc': 0.8691563010215759,
 'val_mcc': 0.7393854840096149,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_5_True_True_0.01_adamw_32_min_mse_0.01_1_135',
 'train_mae': 4.7501335,
 'val_mae': 8.544613,
 'test_mae': 8.676856,
 'train_mse': 61.469128,
 'val_mse': 135.64902,
 'test_mse': 140.35268,
 'train_mcc': 0.8905810254413973,
 'test_mcc': 0.7558911097519115,
 'test_acc': 0.8777992725372314}

In [18]:
all = res_test
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_2] for k in all])
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 5,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 135,
 'train_loss': 0.09937081,
 'val_mse_age': 136.1627,
 'train_acc': 0.9453597068786621,
 'val_acc': 0.8691563010215759,
 'val_mcc': 0.7393854840096149,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_5_True_True_0.01_adamw_32_min_mse_0.01_1_135',
 'train_mae': 4.7501335,
 'val_mae': 8.544613,
 'test_mae': 8.676856,
 'train_mse': 61.469128,
 'val_mse': 135.64902,
 'test_mse': 140.35268,
 'train_mcc': 0.8905810254413973,
 'test_mcc': 0.7558911097519115,
 'test_acc': 0.8777992725372314}

## 4

In [6]:
resnet_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
])

dict_model = dict(
    # dictionary with model information
    out_channels=[2],
    transforms=[
        (
            '1',
            resnet_transform,
            resnet_transform,
            True,
        ),
    ]
)

list_model = [dict(zip(dict_model.keys(), k)) for k in itertools.product(*dict_model.values())]

In [7]:
if do_train:
    for d in tqdm(list_model):
        set_seed(seed)
        
        d = d.copy()
        transforms = d.pop('transforms')

        train(
            model=ResNetClassifier(**d),
            dict_model=d,
            log_dir="./logs_full_4",
            data_path="../data_full",
            save_path="./models/saved_full_4",
            lr=1e-2,
            optimizer_name="adamw",
            n_epochs=150,
            batch_size=32,
            num_workers=3,
            scheduler_mode='min_mse',
            debug_mode=False,
            device=None,
            steps_save=15,
            use_cpu=False,
            loss_age_weight=1e-2,
            scheduler_patience=50,
            train_transforms=transforms[1],
            test_transforms=transforms[2],
            suffix=transforms[0],
            use_cache=transforms[3],
        )

  0%|          | 0/1 [00:00<?, ?it/s]


NameError: name 'set_seed' is not defined

### Results

In [ ]:
res_test = test(
    data_path = "../data_full",
    save_path = './models/saved_full_4',
    n_runs = 1,
    batch_size = 64,
    num_workers = 0,
    debug_mode = False,
    use_cpu = False,
    save = True,
    verbose = False,
    transforms=resnet_transform,
    use_cache=True,
)

100%|██████████| 40/40 [30:18<00:00, 45.47s/it]  


In [ ]:
all = res_test
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_1] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 5,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 135,
 'train_loss': 0.09937081,
 'val_mse_age': 136.1627,
 'train_acc': 0.9453597068786621,
 'val_acc': 0.8691563010215759,
 'val_mcc': 0.7393854840096149,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_5_True_True_0.01_adamw_32_min_mse_0.01_1_135',
 'train_mae': 4.7501335,
 'val_mae': 8.544613,
 'test_mae': 8.676856,
 'train_mse': 61.469128,
 'val_mse': 135.64902,
 'test_mse': 140.35268,
 'train_mcc': 0.8905810254413973,
 'test_mcc': 0.7558911097519115,
 'test_acc': 0.8777992725372314}

In [ ]:
all = res_test
# ascending order
sort_idx = np.argsort([k['dict'][metric_filter_2] for k in all])
all[sort_idx[0]]['dict']

{'in_channels': 3,
 'out_channels': 2,
 'dim_layers': [32, 64, 128],
 'block_conv_layers': 5,
 'residual': True,
 'max_pooling': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_batch_size': 32,
 'train_scheduler_mode': 'min_mse',
 'train_loss_age_weight': 0.01,
 'train_suffix': '1',
 'epoch': 135,
 'train_loss': 0.09937081,
 'val_mse_age': 136.1627,
 'train_acc': 0.9453597068786621,
 'val_acc': 0.8691563010215759,
 'val_mcc': 0.7393854840096149,
 'model_class': 'cnn',
 'path_name': '3_2_[32_64_128]_5_True_True_0.01_adamw_32_min_mse_0.01_1_135',
 'train_mae': 4.7501335,
 'val_mae': 8.544613,
 'test_mae': 8.676856,
 'train_mse': 61.469128,
 'val_mse': 135.64902,
 'test_mse': 140.35268,
 'train_mcc': 0.8905810254413973,
 'test_mcc': 0.7558911097519115,
 'test_acc': 0.8777992725372314}